# k-means Clustering

The k-means clustering algorithm is a method of vector quantization which partitions n data points into k clusters, in which each data point belongs to the cluster with the nearest mean (cluster center).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%pwd
%cd 'drive'
%cd 'My Drive'
%cd 'MLResearchProject'
%cd 'Data_Sets'

/content/drive
/content/drive/My Drive
/content/drive/.shortcut-targets-by-id/1vm2aLzdpci8cKMrtS3X1S57oTdxizmWG/MLResearchProject
/content/drive/.shortcut-targets-by-id/1vm2aLzdpci8cKMrtS3X1S57oTdxizmWG/MLResearchProject/Data_Sets


In [ ]:
import csv
import pandas as pd
import numpy as np

filename='Questions.csv'

#Read csv
pdf = pd.read_csv(filename)
pdf = pdf.drop(columns=['Answer 1', 'Answer 2', 'Answer 3', 'Answer 4', 'Answer 5', 'Explanation', 'Author'], axis=1)
pdf

,Question
0,The runtime for the following code fragment is...
1,"Given the binary search algorithm, as taught i..."
2,An algorithm has time complexity . Using the D...
3,Which one of the following sorting algorithms ...
4,Given the tree: a / \ b c / \ / \ e f g hWhat ...
...,...
1073,2 functions refer to the run-times of 2 algori...
1074,Which of the following sorts has the quickest ...
1075,You plan to store data in a closed hash table ...
1076,Which of the following is the correct definiti...


##### Importing packages

In [ ]:
# Importing packages
import re
import os
import sys
import string
import codecs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score


#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.metrics.pairwise import euclidean_distances
#from sklearn.metrics.pairwise import cosine_similarity
#from scipy.cluster.hierarchy import ward, dendrogram
#from scipy.spatial.distance import cosine
#from scipy.stats.stats import pearsonr

# For text pre-processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

### Text Preprocessing

In [ ]:
# Removing punctuation
pdf['Questions_P1'] = pdf['Question'].map(lambda x: re.sub('[,\.!?:></="-;]', ' ', x))

# Tokenizing
tokenizer = RegexpTokenizer(r'\w+')
pdf['Questions_P2'] = pdf['Questions_P1'].apply(lambda x: tokenizer.tokenize(x.lower()))

# Stop words removal
nltk.download('stopwords')
stop = stopwords.words('english')
stop.append('nbsp')
pdf['Questions_P2'] = pdf['Questions_P2'].apply(lambda x: [item for item in x if item not in stop])

# Stemming using Porter
#ps = PorterStemmer()
#def word_stemmer(text):
#    stem_text = ' '.join([ps.stem(i) for i in text])
#    return stem_text

#pdf['Questions_P3'] = pdf['Questions_P2'].apply(lambda x: word_stemmer(x))

# Lemmatizing
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    stem_text = ' '.join([lemmatizer.lemmatize(i) for i in text])
    return stem_text

pdf['Questions_P3'] = pdf['Questions_P2'].apply(lambda x: word_lemmatizer(x))

# Removing short words
pdf['Questions_P'] = pdf['Questions_P3'].map(lambda x: re.sub(r'\b\w{1,2}\b', '', x))
pdf['Questions_P'].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0    runtime following code fragment     int  int  ...
1    given binary search algorithm taught lecture a...
2    algorithm time complexity using definition big...
3    one following sorting algorithm quickest almos...
4                given tree      hwhat order traversal
Name: Questions_P, dtype: object

In [ ]:
pdf = pdf.drop(columns=['Questions_P1', 'Questions_P2', 'Questions_P3'], axis=1)
pdf.head()

,Question,Questions_P
0,The runtime for the following code fragment is...,runtime following code fragment int int ...
1,"Given the binary search algorithm, as taught i...",given binary search algorithm taught lecture a...
2,An algorithm has time complexity . Using the D...,algorithm time complexity using definition big...
3,Which one of the following sorting algorithms ...,one following sorting algorithm quickest almos...
4,Given the tree: a / \ b c / \ / \ e f g hWhat ...,given tree hwhat order traversal


In [ ]:
text_list1 = pdf['Questions_P']

### Building k-means model

In [ ]:
# Build k-means model with Tf-idf vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

In [ ]:
X = vectorizer.fit_transform(text_list1).toarray()

true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

# Needs visualisation of clusters
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print


Top terms per cluster:
Cluster 0:
 following
 statement
 algorithm
 sorting
 true
 false
 graph
 stable
 data
 matrix
Cluster 1:
 int
 complexity
 time
 case
 algorithm
 worst
 average
 best
 following
 sort
Cluster 2:
 hash
 table
 closed
 probing
 linear
 load
 open
 bucket
 step
 element
Cluster 3:
 tree
 traversal
 binary
 avl
 order
 node
 following
 balanced
 output
 search
Cluster 4:
 list
 sort
 array
 linked
 element
 following
 sorted
 search
 stable
 sorting


In [ ]:
print("Prediction")

Y = vectorizer.transform(["Hash table question"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["This statement has false complexity"])
prediction = model.predict(Y)
print(prediction)
 

Prediction
[2]
[0]
